In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

data = pd.read_csv('SBAnational.csv', sep = ',', encoding = 'latin1')
print(data.head())

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


   LoanNr_ChkDgt                           Name          City State    Zip  \
0     1000014003                 ABC HOBBYCRAFT    EVANSVILLE    IN  47711   
1     1000024006    LANDMARK BAR & GRILLE (THE)     NEW PARIS    IN  46526   
2     1000034009          WHITLOCK DDS, TODD M.   BLOOMINGTON    IN  47401   
3     1000044001  BIG BUCKS PAWN & JEWELRY, LLC  BROKEN ARROW    OK  74012   
4     1000054004    ANASTASIA CONFECTIONS, INC.       ORLANDO    FL  32801   

                            Bank BankState   NAICS ApprovalDate ApprovalFY  \
0               FIFTH THIRD BANK        OH  451120    28-Feb-97       1997   
1                1ST SOURCE BANK        IN  722410    28-Feb-97       1997   
2        GRANT COUNTY STATE BANK        IN  621210    28-Feb-97       1997   
3  1ST NATL BK & TR CO OF BROKEN        OK       0    28-Feb-97       1997   
4        FLORIDA BUS. DEVEL CORP        FL       0    28-Feb-97       1997   

   Term  NoEmp  NewExist  CreateJob  RetainedJob  FranchiseCod

In [4]:
#Removing $ and , from columns
data[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']] = data[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].replace({"\$" : '', ',': ''}, regex = True)

#turning int columns into floats

data['DisbursementGross'] = [float(i) for i in data['DisbursementGross']]
data['BalanceGross'] = [float(i) for i in data['BalanceGross']]
data['ChgOffPrinGr'] = [float(i) for i in data['ChgOffPrinGr']]
data['GrAppv'] = [float(i) for i in data['GrAppv']]
data['SBA_Appv'] = [float(i) for i in data['SBA_Appv']]

#looking at only VA from the whole Dataset
x= data.loc[data['State'] == 'VA']
va = x.copy()
vaDef = data.copy()

#creating a Default col
va['Default'] = np.where(va['MIS_Status'] == 'P I F', 0, 1)

In [5]:
#creating defaults dataframe
naics_df= va.loc[(va['NAICS'] > 0 )& (va['Default'] ==1)]
naics = [int(str(i)[:2]) for i in naics_df['NAICS']]
defaults = pd.DataFrame(naics, naics_df['Default'].values)
defaults = defaults.reset_index()
defaults.columns = ['Default', 'NAICS']
defaults = defaults.groupby('NAICS').count()

#seeing total in each industry 
naics_df2= va.loc[va['NAICS'] > 0 ]
naics2 = [int(str(i)[:2]) for i in naics_df2['NAICS']]

total_industry = pd.DataFrame(naics2, naics_df2['Default'].values)
total_industry = total_industry.reset_index()
total_industry.columns = ['Default', 'NAICS']
total_industry = total_industry.groupby('NAICS').count()


In [6]:
#print(defaults, total_industry)
#creating default rate percentage
a = defaults['Default'].values 
b = total_industry['Default'].values 

defRate = []
for i in range(len(a)):
    defRate.append(a[i]/b[i])

    
defaults['IndustrySize'] = total_industry['Default']
defaults['DefaultRate'] = defRate
defaults =  defaults.sort_values(by='DefaultRate', ascending = False)
print(defaults)

       Default  IndustrySize  DefaultRate
NAICS                                    
22           5            12     0.416667
48         108           347     0.311239
...        ...           ...          ...
11           8            50     0.160000
62          83           865     0.095954

[22 rows x 3 columns]


In [7]:
#next hypothesis: Bigger the totals the less the default rate 
pd.options.display.float_format = '{:20,.2f}'.format
va['DisbursementGross'].describe()

count              13,264.00
mean              197,348.31
                ...         
75%               236,005.50
max             4,869,000.00
Name: DisbursementGross, Length: 8, dtype: float64

In [8]:
loansize = va.copy()
#loansize['loanGroup'] = pd.cut(loansize.DisbursementGross, [35000, 98186, 236005, np.inf], labels = ['Q1', 'Q2', 'Q3'])


In [9]:
#Sorting gross loan size by groups
loansize['loanGroup'] = np.where(loansize['DisbursementGross'] > 236005, 'Large', (np.where(loansize['DisbursementGross'] < 35000, 'Small', 'Medium')))
grossInd = loansize[['loanGroup', 'Default']].groupby('loanGroup', as_index = False).count().copy()
grossInd.columns = ['loanGroup', 'IndustrySize']
#print(grossInd)

#how many in each group defaulted 
sizeCompD = loansize[['loanGroup', 'Default']].groupby('loanGroup', as_index = False).apply(lambda col: col[col['Default'] == 1]).copy()
sizeD = sizeCompD.groupby('loanGroup', as_index = False).count()
#print(sizeCompD.groupby('loanGroup', as_index = False).count())

#calculating default rate for size of loan 
sx = grossInd['IndustrySize'].values
sd = sizeD['Default'].values

sizeD['TotalLoan'] = grossInd['IndustrySize']

sdrate = [ sd[i]/ sx[i] for i in range(len(sx))]
sizeD['Default Rate'] = sdrate 

sizeD = sizeD.sort_values(by='Default Rate', ascending = False)
print('Small: < $35000, Medium: < $98000, Large: > $98000')
print(sizeD)

Small: < $35000, Medium: < $98000, Large: > $98000
  loanGroup  Default  TotalLoan         Default Rate
2     Small     1113       3238                 0.34
1    Medium     1070       6710                 0.16
0     Large      361       3316                 0.11


In [10]:
#industry analysis: Services 54
industry_analysis = va.loc[va['NAICS'] > 0].copy()
industry_analysis['NAICS'] = [int(str(i)[:2]) for i in industry_analysis['NAICS']]
industry_analysis = industry_analysis.loc[industry_analysis['NAICS'] == 54]
industry_analysis.shape

(1472, 28)

In [11]:
del industry_analysis['ChgOffDate']
industry_analysis = industry_analysis.dropna()

In [12]:
#Will not run anymore
#industry_analysis['Year'] = [int(i) for i in industry_analysis['YearAppv']]
#del industry_analysis['YearAppv']

#industry_analysis.head()

In [18]:
industry_analysis['ApprovalFY'] = [int(i) for i in industry_analysis['ApprovalFY']]
#if loan was given during recession years
industry_analysis['Recession'] = np.where((industry_analysis['ApprovalFY'] > 2007) & (industry_analysis['ApprovalFY'] <= 2009), 1, 0)
#loans with terms greater than 240 are related to real estate 
industry_analysis['RealEstate'] = np.where(industry_analysis['Term'] >= 240, 1, 0)
#If a firm is new or an existing business 
industry_analysis['New'] = np.where(industry_analysis['NewExist'] == 2, 1, 0)

In [14]:
industry_analysis.head()

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv,Default,Recession,RealEstate,New
1210,1008336002,VICKY VALENTINE DESIGN,GLEN ALLEN,VA,23059,BBCN BANK,CA,54,8-Feb-06,2006,25,1,1.00,0,1,1,2,0,N,28-Feb-06,"5,000.00",0.00,CHGOFF,"2,657.00","5,000.00","4,250.00",1,0,0,0
1971,1013175000,JettNet Technical Services Inc,WOODBRIDGE,VA,22193,BANK OF AMERICA NATL ASSOC,NC,54,4-Jun-04,2004,84,4,1.00,0,0,1,2,Y,N,31-Jul-04,"30,000.00",0.00,P I F,0.00,"30,000.00","15,000.00",0,0,0,0
1977,1013225002,"IT Shows, Inc.",ALEXANDRIA,VA,22312,MANUFACTURERS & TRADERS TR CO,MD,54,4-Jun-04,2004,60,1,1.00,0,0,1,1,Y,N,30-Sep-05,"99,600.00",0.00,P I F,0.00,"50,000.00","25,000.00",0,0,0,0
2151,1014495004,"JJ AUTOMATION, LLC",CHARLOTTESVILLE,VA,22911,BANK OF AMERICA NATL ASSOC,NC,54,14-Jun-04,2004,84,1,1.00,0,0,1,2,Y,N,30-Nov-04,"30,000.00",0.00,P I F,0.00,"30,000.00","15,000.00",0,0,0,0
2153,1014525000,Deligon Interactive,FREDERICKSBURG,VA,22407,BANK OF AMERICA NATL ASSOC,NC,54,14-Jun-04,2004,84,4,2.00,0,0,1,2,N,N,31-Oct-05,"10,000.00",0.00,P I F,0.00,"10,000.00","5,000.00",0,0,0,1


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

ida = industry_analysis[['GrAppv', 'SBA_Appv', 'Recession', 'RealEstate', 'Default']]

X = ida[['GrAppv', 'SBA_Appv', 'Recession', 'RealEstate']]
y = ida[['Default']]

X_train, x_test, Y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 0)

clf = LogisticRegression()

In [16]:
clf.fit(X_train, Y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [17]:
clf.predict(x_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
clf.score(x_test, y_test)

0.8076923076923077